In [2]:
import numpy as np
from collections import Counter
import math
import pandas as pd

### Question No 1

In [3]:
test_dict = {"<10": 0.08, "10-15": 0.27, "15-20": 0.10,
             "20-25":0.11,"25-30":0.15,"30-35":0.20,"35-37":0.07,
             "37+":0.02}

In [4]:
def truckSim(n,d):
    truck_levels = [] # Initialize an empty list
    for j in range(n): # Run this loop for n trucks
        cumulative_prob = 0 # Initialize a cumulative probability
        random_num = np.random.uniform(0,1) # Generate a number between 0 and 1
        for (k,v) in d.items(): # Run this loop for each value in the dictionary
            cumulative_prob += v # Add the probability of this particular dictionary entry
            if cumulative_prob > random_num: # Once the cumulative probability is higher than the random number...
                truck_levels.append(k) # ...add the key to the list
                break # and break the for loop
    return truck_levels

### Question No 2

In [5]:
simlist = truckSim(1000,test_dict)

In [6]:
def truck1CI(alpha,keyname,n,simlist):
    finallist = []
    deltalist = []
    for i in range(n):
        templist = np.random.choice(simlist,len(simlist))
        pos = Counter(templist)[keyname]
        finallist.append(pos/len(templist))
    samplemean = np.mean(finallist)
    stderror = np.std(finallist)/np.sqrt(n)
    for i in range(n):
        bs =np.random.choice(finallist,len(simlist))
        deltalist.append(np.mean(bs)-samplemean)
    deltalist = sorted(deltalist,reverse=False)
    ub = deltalist[int(alpha*100)]
    lb = deltalist[int(1-alpha*100)]
    return pd.Series([samplemean,samplemean+lb,samplemean+ub,stderror],index = ['Est.Value','CI Lower Bound',
                                                                                'CI Upper Bound','Standard Error'])

In [7]:
df = pd.DataFrame([[1-0.1, '30-35', 100],[1-0.1, '30-35', 1000],[1-0.1, '30-35', 2500],
                   [1-0.1, '30-35', 5000],[1-0.1, '<10', 100],[1-0.1, '<10', 1000],
                   [1-0.1, '<10', 2500],[1-0.1, '<10', 5000]],
                  columns=['Confid.Level','Truck Fill Level','No of Bootstraps'])
df['Est.Value'],df['CI Lower Bound'],df['CI Upper Bound']= np.zeros(8),np.zeros(8),np.zeros(8)
df2= df.apply(lambda x:(truck1CI(x[0],x[1],x[2],simlist)),axis=1)
df['Est.Value'] = df2['Est.Value']
df['CI Lower Bound']= df2['CI Lower Bound']
df['CI Upper Bound']= df2['CI Upper Bound']
df['Standard Error']= df2['Standard Error']

In [8]:
df

,Confid.Level,Truck Fill Level,No of Bootstraps,Est.Value,CI Lower Bound,CI Upper Bound,Standard Error
0,0.9,30-35,100,0.197160,0.196669,0.197727,0.001225
1,0.9,30-35,1000,0.197183,0.197724,0.196678,0.000394
2,0.9,30-35,2500,0.197119,0.197812,0.196402,0.000250
3,0.9,30-35,5000,0.196715,0.197584,0.195899,0.000178
4,0.9,<10,100,0.074180,0.073768,0.074493,0.000839
5,0.9,<10,1000,0.073913,0.074271,0.073575,0.000261
6,0.9,<10,2500,0.073958,0.074431,0.073510,0.000162
7,0.9,<10,5000,0.073903,0.074439,0.073352,0.000117


### Question No 3

In [11]:
simlist = truckSim(4989,test_dict)

In [19]:
dfQ3 = pd.DataFrame([[1-0.05, '35-37', 200],[1-0.05, '37+', 200],[1-0.1, '35-37', 200],
                   [1-0.1, '37+', 200]],
                  columns=['Confid.Level','Truck Fill Level','No of Bootstraps'])
dfQ3['Est.Value'],dfQ3['CI Lower Bound'],dfQ3['CI Upper Bound']= np.zeros(4),np.zeros(4),np.zeros(4)
df2 = dfQ3.apply(lambda x:(truck1CI(x[0],x[1],x[2],simlist)),axis=1)
dfQ3['Est.Value'] = df2['Est.Value']
dfQ3['CI Lower Bound']= df2['CI Lower Bound']
dfQ3['CI Upper Bound']= df2['CI Upper Bound']
dfQ3['Standard Error']= df2['Standard Error']

In [20]:
dfQ3

,Confid.Level,Truck Fill Level,No of Bootstraps,Est.Value,CI Lower Bound,CI Upper Bound,Standard Error
0,0.95,35-37,200,0.072623,0.072628,0.072623,0.000275
1,0.95,37+,200,0.019266,0.019270,0.019267,0.000124
2,0.90,35-37,200,0.072842,0.072855,0.072842,0.000251
3,0.90,37+,200,0.019196,0.019199,0.019190,0.000139
